## Topicmodelling
used codes from: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("../data/tweets_public.csv", encoding = "utf-8", sep = ",")

In [12]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [13]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/borbota/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/borbota/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [16]:
import random
text_data = []
for line in df["text"]:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        text_data.append(tokens)

In [17]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [18]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

(0, '0.078*"SCREEN_NAME" + 0.032*"flight" + 0.024*"service" + 0.017*"customer" + 0.017*"since" + 0.017*"dallas" + 0.017*"going"')
(1, '0.082*"SCREEN_NAME" + 0.023*"pilot" + 0.016*"class" + 0.016*"flying" + 0.016*"delay" + 0.016*"flight" + 0.009*"happen"')
(2, '0.157*"SCREEN_NAME" + 0.053*"flight" + 0.029*"delay" + 0.026*"thanks" + 0.016*"thank" + 0.010*"horrible" + 0.010*"issue"')
(3, '0.083*"SCREEN_NAME" + 0.038*"flight" + 0.027*"still" + 0.021*"waiting" + 0.014*"first" + 0.014*"number" + 0.014*"attendant"')
(4, '0.078*"SCREEN_NAME" + 0.032*"flight" + 0.024*"customer" + 0.017*"today" + 0.017*"problem" + 0.017*"luggage" + 0.017*"service"')


In [19]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

#pyLDAvis.save_html(lda_display, 'lda.html')

/home/borbota/anaconda/envs/uab/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
